# GAIA PSF Simulation

First attempt to simulate close binary stars observation using the gaia telescope. The latter is imagined as a rectangular pupil, with no pixel binning or aberrations

#### Initialization

In [1]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from matplotlib import pyplot as plt
from simulator import GaiaTelescopeV0

/home/pietrof/miniforge3/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


## The simulator

The idea of the `SimulatorV0` is to simply create a map of close point-like sources, spanning different angles and separations, to convolve with a constructed (very simplified) _Gaia_ PSF (created with `Poppy`)

```python
from scipy.ndimage import zoom

def resample_sky_to_detector(sky_map, sky_pixscale, detector_shape, detector_pixscale):
    """
    Resample the sky map to match the detector's pixel scale and shape.
    """
    scale_factor = sky_pixscale / detector_pixscale
    zoom_factors = (detector_shape[0] / sky_map.shape[0], detector_shape[1] / sky_map.shape[1])
    resampled_sky = zoom(sky_map, zoom_factors, order=1)
    return resampled_sky

# Example usage:
sky_pixscale = 0.01  # arcsec/pixel (sky map)
detector_pixscale = 0.05  # arcsec/pixel (poppy)
detector_shape = (128, 128)
resampled_sky = resample_sky_to_detector(sky_map, sky_pixscale, detector_shape, detector_pixscale)

In [2]:
gt = GaiaTelescopeV0()
gt

           Gaia Telescope V0
           -----------------
          Aperture: 1.5 m x 0.5 m
    Scansion speed: 59.96 arcsec / s
    Orbital Period: 6.0 h
 Precession period: 63.0 d
Wavelength for PSF: 5.5e-07 m

          e2v™ CCD91-72
          -------------
            Band: Gaia_G
      Pixel area: 300.0 um2
     Pixel Scale: 59.0 x 177.0 mas / pix
        CCD fov : 4500 x 1966  pix
               -> 4.42 x 5.80 arcmin
Integration time: 4.42 s
    

In [19]:
cube = gt.ccd.create_raw_binary_cube(13,13,35, gt.ccd.pixel_area*4500*7, 4.42)